In [ ]:
##IMPORTING ALL THE REQUIRED LIBRARIES AND METHODS
from sklearn import metrics
from sklearn.grid_search import RandomizedSearchCV,GridSearchCV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
## Reading data into pandas dataframe
df = pd.read_csv(r'C:\Users\SVR\Documents\Customer Churn\data.csv')

In [ ]:
cols_not_needed = ['State','Account Length','Area Code','Phone']
df = df.drop(cols_not_needed,axis = 1)

In [ ]:
# Converting categorical variable into numerical variable
def cat_num(label):
    if label == "False.":
        return 0
    else:
        return 1
Churn= df['Churn?'].apply(cat_num)

In [ ]:
df['Churn_int'] = Churn
df.drop(['Churn?'],axis = 1,inplace = True)

In [ ]:
# Converting categorical variables (Int'l Plan,VMail Plan)into numerical variables
mapping_dict = {
    "Int'l Plan":{
        'yes' : 1,
        'no' : 0
        },
    "VMail Plan":{
        'yes' : 1,
        'no' : 0
        }
    }


df = df.replace(mapping_dict)    

In [ ]:
# selecting features and target variable
columns = df.columns.tolist()
features = df.columns[:-1].tolist()
label = df.columns[-1]


In [ ]:

X = df.iloc[:,0:-1]
y = df.Churn_int
X.head()

In [ ]:
#float colummns
float_cols = X.select_dtypes(['float']).columns.tolist()

In [ ]:
# let us plot histogram of features to understand their distribution
fig = plt.figure(figsize = (12,12))

for i in range(0,len(float_cols)):
    ax = fig.add_subplot(4,2,i+1)
    col = float_cols[i]
    ax.hist(df[col])
    plt.title(col)
    


In [ ]:
#summary of all the features
df.describe()

In [ ]:
from sklearn import preprocessing
pre = preprocessing.StandardScaler()

# standardizing features
np_scaled = pre.fit_transform(X)
df_normalized = pd.DataFrame(np_scaled,columns = features)
df_normalized.head()



In [ ]:
X = df_normalized
y = df.Churn_int

In [ ]:
# splitting entire data into train and test sets
from sklearn.cross_validation import train_test_split
X=df_normalized.iloc[:,:-1]
y = df.Churn_int
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0)

In [ ]:
# importing all the required classifiers to model the problem at hand
from sklearn.cross_validation import cross_val_score,KFold,cross_val_predict
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.linear_model import LogisticRegression as LR
from sklearn import svm
from sklearn import metrics
###kf = KFold(X.shape[0],random_state = 1)
logreg = LR()
knn     = KNN()
rf  = RF()
support_vector_machine  = svm.SVC()

classifiers = ['Logistic_Regression','KnearestNeighbors','RandomForest','Support Vector Machine']
estimators_ojects = [logreg,knn,rf,support_vector_machine]
'''scores = cross_val_score(logreg,X,y,cv=3)
scores = pd.Series(scores)
scores.mean()'''

In [ ]:
# function to calculate accuracies of each model which is fitted to the data
def accuracy_using_cv(estimators_objects):
    accuracies = []
    for est in estimators_objects:
        kf = KFold(X.shape[0], random_state=1)
        scores = cross_val_score(est,X,y,cv = kf)
        scores = pd.Series(scores).mean()
        #scores = scores.mean()
        accuracies.append(scores)
        print('The Avg Accuracy of classifier %s using 3 fold CV is %f' %(est,scores),'\n \n')
    return accuracies

accuracy = accuracy_using_cv(estimators_ojects)
accuracy
index = accuracy.index(max(accuracy))
print('Classifier %s has highest accuarcy with %f' %(classifiers[index],max(accuracy)) )


        

In [ ]:
# null accuracy ,.ie., predicting all the labels as the one which has majority
y_test.value_counts().head(1)/len(y_test)


In [ ]:
# function to find confusion matrices,Precision,Recall and F1 scores of the classifiers
from collections import defaultdict
confusion_matrices = defaultdict(dict)
recall_scores = {}
accuracy_scores = {}
f1_scores = {}
auc = {}
for est in estimators_ojects:
    est.fit(X_train,y_train)
    y_pred_class = est.predict(X_test)
    matrix = metrics.confusion_matrix(y_test,y_pred_class)
    recall_score = metrics.recall_score(y_test,y_pred_class)
    accuracy = metrics.accuracy_score(y_test,y_pred_class)
    f1_score = metrics.f1_score(y_test,y_pred_class)
    aucs = metrics.roc_auc_score(y_test,y_pred_class)
    confusion_matrices[classifiers[estimators_ojects.index(est)]] = matrix
    recall_scores[classifiers[estimators_ojects.index(est)]] = recall_score
    accuracy_scores[classifiers[estimators_ojects.index(est)]] = accuracy
    f1_scores[classifiers[estimators_ojects.index(est)]] = f1_score
    auc[classifiers[estimators_ojects.index(est)]] = aucs
    

In [ ]:
# installing seaborn library for plotting
!conda install seaborn

In [ ]:
# importing seaborn library and plotting confusion matrices which tells us about true positives,true negatives,false positives,false negatives
import seaborn as sns
fix, ax = plt.subplots(figsize=(16, 12))
plt.suptitle('Confusion Matrix of Various Classifiers')
i =1 
for k,val in confusion_matrices.items():
    plt.subplot(2,2,i)
    plt.title(k)
    sns.heatmap(val,annot=True,  fmt='')
    i = i+1
    #sns.plt

In [ ]:
##classifier with max recall score
max(recall_scores, key = lambda x: recall_scores.get(x) )

In [ ]:
##classifier with max accuracy score
max(accuracy_scores,key = lambda x : accuracy_scores.get(x))

In [ ]:
f1_scores


For a good classifier F1 score (HM of precison and recall) is 1.We can take out Logistic Regression classifier as it has only 0.23 F1 score

In [ ]:
auc


For a good classifier AUC should have max value,here RF performs well.For all the above metrics,RF is consistently performing well.So,will choose RF as our final model and let us try to build a best RF classfier by tuning the parameters of it.

In [ ]:
#Implementation of RF

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


rf = RF()
rf_fit = rf.fit(X_train, y_train)
y_pred_test = rf_fit.predict(X_test)

print ('CONFUSION MATRIX :\n\n',metrics.confusion_matrix(y_test,y_pred_test),'\n\n')
print ('ACCURACY SCORE :',metrics.accuracy_score(y_test,y_pred_test),'\n\n')
print ('CLASSIFICATION REPORT:\n\n',metrics.classification_report(y_test,y_pred_test))

In [ ]:
parameter_grid = { 
    'n_estimators': [i for i in range(200,1,-10)],
    'max_features': ['auto', 'sqrt', 'log2']
}
#scores = ['precision', 'recall']

Let us tune the hyperparameters of Random Forest classifier using GridSearch technique, which takes the set of all possible parameters combination given to it

In [ ]:
from sklearn.grid_search import GridSearchCV,RandomizedSearchCV
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RF
rf = RF()
#kf = KFold(len(X), n_folds=10,shuffle=True, random_state=1)
rf_gs = GridSearchCV(estimator=rf,param_grid = parameter_grid,cv = 10,scoring = 'accuracy',n_jobs=-1)
rf_gs.fit(X,y)


In [ ]:
rf_gs.grid_scores_

In [ ]:
mean_scores = [result.mean_validation_score for result in rf_gs.grid_scores_]
mean_scores

In [ ]:
print(rf_gs.best_score_)
print(rf_gs.best_params_)
print(rf_gs.best_estimator_)

In [ ]:
# printing accuracy score,roc_auc_score,classification report for the classifier with the best set of parameters selected by the grid search
pred = rf_gs.predict(X_test)
print(metrics.confusion_matrix(y_test,pred))
print()
print(metrics.accuracy_score(y_test,pred))
print()
print(metrics.classification_report(y_test,pred))
print()
print(metrics.roc_auc_score(y_test,pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
rf = RandomForestClassifier(n_estimators = 160,max_features = 'log2',n_jobs=-1)

In [ ]:
rf.fit(X,y)

In [ ]:
importances = rf.feature_importances_

In [ ]:
importances

In [ ]:
rf.score(X,y)

In [ ]:
# let us perfor random search for the best set of paramters
paramet_grid = { 
    'n_estimators': [i for i in range(300,1,-10)],
    'max_features': ['auto', 'sqrt', 'log2']
}
#scores = ['precision', 'recall']
rf_rs = RandomizedSearchCV(rf,paramet_grid,cv = 10,scoring = 'accuracy',n_jobs = -1,n_iter = 10,random_state = 4)
rf_rs.fit(X,y)
rf_rs.grid_scores_

In [ ]:
print(rf_rs.best_score_)
print(rf_rs.best_params_)
print(rf_rs.best_estimator_)

In [ ]:
#!conda install graphviz

In [ ]:
'''export_graphviz(rf,
                feature_names=X.columns,
                filled=True,
                rounded=True)'''




In [ ]:
#import os
#os.system('dot -Tpng tree.dot -o tree.png')

In [ ]:
#print("Features sorted by their score:")
#print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), 
             reverse=True))